# 실행 전 requirements.txt 터미널에서 받아주세요!

### 버스 노선 데이터 노선별 이용량 추출
교통카드빅데이터 통합 정보 시스템: https://stcis.go.kr/pivotIndi/wpsPivotIndicator.do?siteGb=P&indiClss=IC03&indiSel=IC0301


In [97]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [98]:
busroute_usage = pd.read_csv(
    filepath_or_buffer="../data/gyeongn_busroute_total.csv"
    )

In [99]:
busroute_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2513 entries, 0 to 2512
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   city_name         2513 non-null   object 
 1   city_code         2513 non-null   int64  
 2   routeid           2513 non-null   object 
 3   routeno           2513 non-null   object 
 4   routetp           2513 non-null   object 
 5   startnodenm       2513 non-null   object 
 6   endnodenm         2513 non-null   object 
 7   startvehicletime  1413 non-null   float64
 8   endvehicletime    1334 non-null   float64
 9   intervaltime      1056 non-null   float64
 10  intervalsattime   159 non-null    float64
 11  intervalsuntime   152 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 235.7+ KB


In [100]:
busroute_usage["city_name"].unique()

array(['창원시', '진주시', '통영시', '사천시', '김해시', '밀양시', '거제시', '양산시', '의령군',
       '함안군', '창녕군', '고성군', '남해군', '하동군', '산청군', '함양군', '거창군', '합천군'],
      dtype=object)

In [101]:
busroute_usage[(busroute_usage["city_name"] == "김해시") & (busroute_usage["routeno"] == "25")]

,city_name,city_code,routeid,routeno,routetp,startnodenm,endnodenm,startvehicletime,endvehicletime,intervaltime,intervalsattime,intervalsuntime
752,김해시,38070,GHB13,25,지선버스,롯데아울렛(기점),롯데워터파크,535.0,2245.0,NaN,NaN,NaN


#### 웹에서 파일 다운로드하는 크롤러 만들기

In [102]:
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import keyboard

In [108]:
city_name = "김해시" ## "창원시"
startnodenm = "롯데아울렛(기점)"
routeno = 25
findtime = 10000

if city_name == "창원시":
    region_button_selector = "#label_48123_48127_48125_48121_48129 > div"
    
# elif city_name == "진주시":
#     region_button_selector = 
    
# elif city_name == "통영시":
#     region_button_selector = 
    
# elif city_name == "사천시":
#     region_button_selector = 
    
elif city_name == "김해시":
    region_button_selector = "#label_48250 > div"
    
# elif city_name == "밀양시":
#     region_button_selector = 
        
# elif city_name == "거제시":
#     region_button_selector = 
    
# elif city_name == "양산시":
#     region_button_selector = 
    
# elif city_name == "의령군":
#     region_button_selector = 
    
# elif city_name == "함안군":
#     region_button_selector = 
    
# elif city_name == "창녕군":
#     region_button_selector = 
    
# elif city_name == "고성군":
#     region_button_selector = 
    
# elif city_name == "남해군":
#     region_button_selector = 
    
# elif city_name == "하동군":
#     region_button_selector = 
    
# elif city_name == "산청군":
#     region_button_selector = 
    
# elif city_name == "함양군":
#     region_button_selector = 

# elif city_name == "거창군":
#     region_button_selector = 
    
# elif city_name == "합천군":
#     region_button_selector = 
    
# else:
#     raise ValueError(f"Unknown city_name: {city_name}")

options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
driver = wb.Chrome(options = options)


## 교통카드빅데이터 종합정보시스템 -> 노선별 이용량 조회 url
driver.get("https://stcis.go.kr/pivotIndi/wpsPivotIndicator.do?siteGb=P&indiClss=IC03&indiSel=IC0301")


## 기간선택 -> 월 클릭
WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#divRdoDate > li:nth-child(2)"))
).click()


## 공간선택 -> 검색명에 city_name 입력 후 검색 클릭
element = WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#searchRouteSpaceNm"))
)
driver.execute_script("arguments[0].scrollIntoView();", element)
driver.execute_script("arguments[0].click();", element)
element.send_keys(f"{city_name}")


## 검색 버튼 클릭
search_button = WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#space4 > li.box_flex > button"))
)
driver.execute_script("arguments[0].scrollIntoView();", search_button)
driver.execute_script("arguments[0].click();", search_button)


## 지역 버튼 클릭 > 지역마다 클릭 버튼이 다름
region_button = WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, region_button_selector))
)
driver.execute_script("arguments[0].scrollIntoView();", region_button)
driver.execute_script("arguments[0].click();", region_button)


## 노선명 입력칸 클릭 -> 노선번호(명) 입력
route_name_input = WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#popupSearchRouteNo"))
)
driver.execute_script("arguments[0].scrollIntoView();", route_name_input)
driver.execute_script("arguments[0].click();", route_name_input)
route_name_input.send_keys(f"{routeno}")


## 검색 버튼 클릭
search_button = WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#route_space1 > li.box_flex > button"))
)
driver.execute_script("arguments[0].scrollIntoView();", search_button)
driver.execute_script("arguments[0].click();", search_button)

## 노선명과 기종점이 같은지 확인
routeno_len = int(WebDriverWait(driver, findtime).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'pop_box'))
).text.split()[2])

try:
    ## 검색결과 한개인 경우
    WebDriverWait(driver, findtime).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#divBusLineList > div.pop_box > table > tbody"))
    )
    if routeno_len == 1:
        check_routeno = driver.find_element(
            By.CSS_SELECTOR, 
            "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(2)"
            ).text

        check_startnodem = driver.find_element(
            By.CSS_SELECTOR, 
            "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(4)"
            ).text.split(" - ")[0]

        ## 노선명과 기종점이 일치할 경우 체크 박스 클릭
        if check_routeno == str(routeno) and check_startnodem == startnodenm:
            check_click = WebDriverWait(driver, findtime).until(
                EC.element_to_be_clickable((
                    By.CSS_SELECTOR, 
                    "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(1) > div > label > div"
                    ))
            )
            driver.execute_script("arguments[0].scrollIntoView();", check_click)
            driver.execute_script("arguments[0].click();", check_click)
            select_button = WebDriverWait(driver, findtime).until(
                EC.element_to_be_clickable((
                    By.CSS_SELECTOR, 
                    "#popupBusLine > div:nth-child(4) > button"
                    ))
            )
            driver.execute_script("arguments[0].scrollIntoView();", select_button)
            driver.execute_script("arguments[0].click();", select_button)
            
            ## 검색결과조회 버튼 클릭
            search_click = WebDriverWait(driver, findtime).until(
                EC.element_to_be_clickable((
                    By.CSS_SELECTOR, 
                    "#btnSearch > button"
                    ))
            )
            driver.execute_script("arguments[0].scrollIntoView();", search_click)
            driver.execute_script("arguments[0].click();", search_click)
            
            ## 경고 메세지 "확인" 클릭
            try:
                WebDriverWait(driver, findtime).until(
                    EC.alert_is_present()
                )
                alert = driver.switch_to.alert
                alert.accept()  # '확인' 클릭
            except Exception as e:
                print(f"팝업 처리 오류: {e}")

## 아래 아직 미완성 코드 ################################################################################################################
    ## 검색결과 여러 개인 경우
    else:         
        for i in range(1, routeno_len+1):
            check_routeno = driver.find_element(
                By.CSS_SELECTOR, 
                f"#divBusLineList > div.pop_box > table > tbody > tr:nth-child({i}) > td:nth-child(2)"
                ).text

            check_startnodem = driver.find_element(
                By.CSS_SELECTOR, 
                f"#divBusLineList > div.pop_box > table > tbody > tr:nth-child({i}) > td:nth-child(4)"
                ).text
        
        
except:
    # 요소가 존재하지 않을 때 수행할 작업
    print("Error : 일치하는 노선이 존재하지 않습니다.")
    
    
    
    
# check_routeno = driver.find_element(By.CSS_SELECTOR, "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(2)").text

#divBusLineList > div.pop_box > table > tbody > tr:nth-child(1) > td:nth-child(2)
#divBusLineList > div.pop_box > table > tbody > tr:nth-child(2) > td:nth-child(2)
#divBusLineList > div.pop_box > table > tbody > tr:nth-child(1)

#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(2)
# if driver.find_element(By.CLASS_NAME, "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(2)").text
#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(2)
#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(4)

yes
Error : 일치하는 노선이 존재하지 않습니다.


In [68]:
check_startnodem.split(" - ")[0]

'롯데아울렛(기점)'